In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas as pd
import netCDF4
import numpy as np
import os
from datetime import datetime, date
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression

In [6]:
Station_list = ['POLESOUT', 'KISSR0.0', 'LZ2', 'FEBIN', 'MBOXSOU', 'MH24000', 
                'FEBOUT', 'L005', 'L008', 'LZ40', 'L004', 'S308C', 'POLE3S', 
                'RITTAE2', 'L007', 'L006', 'PELBAY3', 'LZ30']
wavelength_array = [400, 412, 442, 490, 510, 560, 620, 665, 674, 681, 709, 754, 865, 940, 1012]

In [3]:
coordinate = pd.read_csv("coordinate.csv")
coordinate["Lat2"] = list(map(lambda lat: int(str(lat)[:2])+ int(str(lat)[2:4])/60 + float(str(lat)[4:])/3600,
                             coordinate["Latitude"]))
coordinate["Lon2"] = list(map(lambda lon: -(int(str(lon)[:2])+ int(str(lon)[2:4])/60 + float(str(lon)[4:])/3600),
                             coordinate["Longtitude"]))
coordinate[["Site Name", "Lat2", "Lon2"]]

,Site Name,Lat2,Lon2
0,POLESOUT,27.038198,-80.918541
1,KISSR0.0,27.141301,-80.846000
2,LZ2,27.189756,-80.828040
3,S133,27.206467,-80.801019
4,S135,27.086633,-80.661156
5,FEBIN,26.959360,-81.069347
6,MBOXSOU,26.921446,-81.059901
7,MH24000,26.877455,-81.018807
8,FEBOUT,26.971805,-81.005937
9,L005,26.956730,-80.972385


In [4]:
def find_closest(array, value):
    min_difference = 9999
    closest_value = None
    for item in array:
        if abs(item-value) < min_difference:
            min_difference = abs(item-value)
            closest_value = item
    return closest_value

In [7]:
all_RS_data = dict()
folder = "/Users/quachhungvietnam/Desktop/Capstone/Sentinel_3_Data/S3_LO/"
files = os.listdir(folder)
for file in files:
    if "S3B" in file:
        path = f"{folder}{file}"
        data = netCDF4.Dataset(path)
        lon_array = np.array(data.variables["lon"])[0]
        lat_array = np.sort([x[0] for x in np.array(data.variables["lat"])])
        date = file[16:24]
        hour = file[25:31]
        for station in Station_list: 
            [[lat, lon]] = np.array(coordinate.loc[coordinate["Site Name"]==station, ["Lat2","Lon2"]])
            closest_lon = find_closest(lon_array, lon)
            closest_lat = find_closest(lat_array, lat)
            x, y = list(lon_array).index(closest_lon), list(lat_array).index(closest_lat)
            all_RS_data[(date, station)] = {}
            for wavelength in wavelength_array:
                all_RS_data[(date, station)][wavelength] = np.array(data.variables[f'Rrs_{wavelength}'])[y,x]
        data.close()